# Training YOLOv8 on Geospatial Dataset

In [2]:
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image
from PIL import Image
import pandas as pd
import numpy as np

## Train YOLOv8 model on Geospatial dataset without tree pretraining

In [3]:
!yolo detect train model=yolov8s.yaml data="/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/data.yaml" workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8s_GEE_no_pretrain

New https://pypi.org/project/ultralytics/8.2.80 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/data.yaml, epochs=100, time=None, patience=50, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8s_GEE_no_pretrain, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer

## Inference on Test Set

In [8]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_GEE_no_pretrain/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/YOLOv8s_GEE_no_pretrain Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))


0: 640x640 18 trees, 12.9ms
1: 640x640 1 tree, 12.9ms
2: 640x640 6 trees, 12.9ms
3: 640x640 10 trees, 12.9ms
4: 640x640 16 trees, 12.9ms
5: 640x640 14 trees, 12.9ms
Speed: 2.3ms preprocess, 12.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


In [9]:
test_images

['/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_15_switzerland_png.rf.7cc73ca9e01e0499401e748e04417980.jpg',
 '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_17_Netherlands_2022_png.rf.febcfa8d68aad81289076e88d421d655.jpg',
 '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_20_Netherlands_2022_png.rf.b4b281ede776dcae8d552ddc879a5538.jpg',
 '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_6_Spain_png.rf.4b400eaec4e4f32e52991f93f885317e.jpg',
 '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_7_Latvia_Infrared_png.rf.c6c2165737270246409b41b0b48efe72.jpg',
 '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_7_Netherlands_2021_png.rf.146db97d35bee6f2829075150e1644f1.jpg']

In [10]:
# Predictions from YOLO model
predictions = [
    18, 1, 6, 10, 16, 14
]

# Ground truth from labels
ground_truth = [
    14, 7, 12, 13, 12, 10
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")



     Image  Prediction  Ground Truth
0  Image 0          18            14
1  Image 1           1             7
2  Image 2           6            12
3  Image 3          10            13
4  Image 4          16            12
5  Image 5          14            10
Mean Absolute Error (MAE): 4.50


## Training Pre-trained tree YOLOv8s normal model

In [11]:
!yolo detect train \
    model="/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_normal/weights/best.pt" \
    data="/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/data.yaml" \
    workers=2 \
    batch=12 \
    device=0 \
    epochs=100 \
    patience=50 \
    name=yolov8s_normal_GEE_pretrain

New https://pypi.org/project/ultralytics/8.2.80 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_normal/weights/best.pt, data=/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/data.yaml, epochs=100, time=None, patience=50, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8s_normal_GEE_pretrain, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=Non

## Inference on Test Set

In [12]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_normal_GEE_pretrain/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/YOLOv8s_GEE_normal_pretrain Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))


0: 640x640 14 trees, 12.9ms
1: 640x640 3 trees, 12.9ms
2: 640x640 6 trees, 12.9ms
3: 640x640 13 trees, 12.9ms
4: 640x640 19 trees, 12.9ms
5: 640x640 13 trees, 12.9ms
Speed: 3.2ms preprocess, 12.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [13]:
# Predictions from YOLO model
predictions = [
    14, 3, 6, 13, 19, 13
]

# Ground truth from labels
ground_truth = [
    14, 7, 12, 13, 12, 10
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")



     Image  Prediction  Ground Truth
0  Image 0          14            14
1  Image 1           3             7
2  Image 2           6            12
3  Image 3          13            13
4  Image 4          19            12
5  Image 5          13            10
Mean Absolute Error (MAE): 3.33


# Training on the Pretrained / Modified YOLOv8-small Model

In [14]:
!yolo detect train \
    model="/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_small_trial/weights/best.pt" \
    data="/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/data.yaml" \
    workers=2 \
    batch=12 \
    device=0 \
    epochs=100 \
    patience=50 \
    name=yolov8s_small_GEE_pretrain

New https://pypi.org/project/ultralytics/8.2.80 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_small_trial/weights/best.pt, data=/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/data.yaml, epochs=100, time=None, patience=50, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8s_small_GEE_pretrain, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf

In [5]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_small_GEE_pretrain/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/YOLOv8s_GEE_small_pretrain Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))


0: 640x640 16 trees, 9.3ms
1: 640x640 6 trees, 9.3ms
2: 640x640 8 trees, 9.3ms
3: 640x640 9 trees, 9.3ms
4: 640x640 10 trees, 9.3ms
5: 640x640 11 trees, 9.3ms
Speed: 28.0ms preprocess, 9.3ms inference, 21.0ms postprocess per image at shape (1, 3, 640, 640)


In [3]:
# Predictions from YOLO model
predictions = [
    16, 6, 8, 9, 10, 11
]

# Ground truth from labels
ground_truth = [
    14, 7, 12, 13, 12, 10
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")



     Image  Prediction  Ground Truth
0  Image 0          16            14
1  Image 1           6             7
2  Image 2           8            12
3  Image 3           9            13
4  Image 4          10            12
5  Image 5          11            10
Mean Absolute Error (MAE): 2.33


In [8]:
for img in test_images:
    print(img)

/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_15_switzerland_png.rf.7cc73ca9e01e0499401e748e04417980.jpg
/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_17_Netherlands_2022_png.rf.febcfa8d68aad81289076e88d421d655.jpg
/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_20_Netherlands_2022_png.rf.b4b281ede776dcae8d552ddc879a5538.jpg
/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_6_Spain_png.rf.4b400eaec4e4f32e52991f93f885317e.jpg
/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_7_Latvia_Infrared_png.rf.c6c2165737270246409b41b0b48efe72.jpg
/home/jupyter/ee_tree_counting/Data/GEE Dataset/Dataset YOLO/test/images/Region_7_Netherlands_2021_png.rf.146db97d35bee6f2829075150e1644f1.jpg
